In [1]:
import extract_feed as e

cli, session_usr = e.bluesky_login()

# to collect information from bluesky, you need a bluesky client. 
# to do that, you need to create a user (with a name, etc) to log into bluesky. the bluesky client session is then tied to this specific user
# for completeness, information on the specific user we are logging in as should also be collected
cli_did = e.get_did(cli, session_usr)
resp = cli.get_profile(actor=cli_did)
cli_username = session_usr
cli_displayname = resp.display_name
cli_account_created_at = resp.created_at
cli_deets = f"{cli_did}|{cli_username}|{cli_displayname}|{cli_account_created_at}"

# before parsing begins, write a control table locally
# this should prevent records already saved in Azure from being ingested again
print(f"New BlueSky feed-extract session opened at {e.datetime.now(e.pytz.timezone("America/New_York")).strftime('%Y-%m-%d %H:%M:%S %Z')}")
print(f"Logging in as BlueSky User {e.USR}... \nLET'S GET THIS DATA! ( ͡⌐■ ͜ʖ ͡-■)\n\n")
watermark_tbl_written = e.write_watermark_table()
if watermark_tbl_written:
    watermark_tbl = e.pd.read_csv(f"{e.WTM_TBL_DIR}/extract_feed_control_tbl.csv").to_dict(orient='list')
else:
    watermark_tbl = None

following_users = {item.handle: [item.did, item.display_name] for item in e.get_following_users(cli, session_usr)}
print(f"Detected {len(following_users):,} BlueSky Users being followed by user @{session_usr}")
print(f"Parsing posts...")
df = None
c = 0
usr = 'bitnerd.gvg.io'
print(f"\n\n{str(c).zfill(3)} of {str(len(following_users)).zfill(3)} | Parsing posts from user @{usr}...")
# accumulate data across the feeds of many users
# stash_user_posts() will save CSV data should it hit the 100 mb threshold mid-ingestion for a single user
df = e.stash_user_posts(cli_deets, schema_input=e.SCHEMA, bsky_client=cli, bsky_did=following_users[usr][0], bsky_username=usr, wtm_tbl=watermark_tbl)

New BlueSky feed-extract session opened at 2025-04-10 11:52:54 EDT
Logging in as BlueSky User thehippus.bsky.social... 
LET'S GET THIS DATA! ( ͡⌐■ ͜ʖ ͡-■)




WARNING! WARNING! WARNING!

Zero CSV files found in Azure Blob directory bluesky_posts/, container sfsandbox
This means incremental ingestion will not be applied. If that is unexpected, then this run may be ingesting duplicate records.
If you don't want that, cancel this ingestion now with CTRL+C!

Detected 88 BlueSky Users being followed by user @thehippus.bsky.social
Parsing posts...


000 of 088 | Parsing posts from user @bitnerd.gvg.io...


In [3]:
f"{len(df.index):,}"

'1,920'